<a href="https://colab.research.google.com/github/spooki-chan/CS-31_SDGP/blob/sub/RecommedationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hybrid Recommendation System


In [1]:
# Importing frameworks
import pandas as pd
import numpy as np
import ast 
from ast import literal_eval

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import warnings; warnings.simplefilter('ignore')



In [2]:
df = pd.read_csv("/content/newfood.csv")

In [3]:
df.head()

,id,title,pricePerServing,weightPerServing,vegetarian,vegan,glutenFree,dairyFree,sustainable,veryHealthy,...,Vitamin D/µg,Vitamin K/µg,Vitamin C/mg,Alcohol/g,Caffeine/g,stars,popularity,userId,tfdbId,recipeId
0,1,Fried Anchovies with Sage,5.6051,226,False,False,False,True,False,False,...,0.29,0.00,0.00,0.0,0.0,3,150.437577,1,1,1915
1,2,Anchovies Appetizer With Breadcrumbs & Scallions,0.8206,33,False,False,False,True,False,False,...,0.00,7.18,0.00,0.0,0.0,1,139.082615,1,2,1916
2,3,"Carrots, Cauliflower And Anchovies",4.3800,364,False,False,False,True,False,True,...,0.00,104.27,32.60,0.0,0.0,4,107.376788,1,3,1918
3,4,Bap Story: Stir Fried Anchovies (Myulchi Bokkeum),8.1122,711,False,False,True,True,False,True,...,6.24,16.02,3.88,0.0,0.0,3,112.312950,1,4,1919
4,5,"Bread, Butter And Anchovies",0.2557,36,False,False,False,False,False,False,...,0.00,2.21,0.00,0.0,0.0,5,43.926995,1,5,1920


Removing Unwanted Columns In the Dataset

In [4]:
df.drop(labels=['vegetarian','vegan', 'glutenFree', 'dairyFree', 'sustainable', 'weightPerServing','pricePerServing', 'gaps', 'lowFodmap', 'ketogenic'],axis=1, inplace=True)
df.drop(labels=['whole30','spoonacularScore', 'Vitamin B5/mg', 'Vitamin B6/mg', 'healthScore','percentProtein', 'percentFat', 'Magnesium/mg', 'Vitamin B12/µg'],axis=1, inplace=True)
df.drop(labels=['percentCarbs','Zinc/mg', 'Manganese/mg', 'Fiber/g', 'Vitamin E/mg','Vitamin B1/mg', 'Folate/µg', 'Potassium/mg', 'Copper/mg','Vitamin A/IU','Vitamin D/µg','Vitamin K/µg'],axis=1, inplace=True)
df.drop(labels=['Fat/g','Saturated Fat/g', 'Carbohydrates/g','Cholesterol/mg','Sodium/mg','Protein/g','Vitamin B3/mg','Selenium/µg','Phosphorus/mg','Iron/mg','Vitamin B2/mg','Calcium/mg','Vitamin C/mg','Alcohol/g','Caffeine/g'],axis=1, inplace=True)


In [5]:
df.head()

,id,title,veryHealthy,veryPopular,readyInMinutes,spoonacularSourceUrl,image,aggregateLikes,dishTypes,ingredients,calories,Sugar/g,stars,popularity,userId,tfdbId,recipeId
0,1,Fried Anchovies with Sage,False,False,45,https://spoonacular.com/fried-anchovies-with-s...,https://spoonacular.com/recipeImages/1-556x370...,1354,Maindish,"anchovies,baking powder,egg,flour,sage leaves,...",384.09,0.17,3,150.437577,1,1,1915
1,2,Anchovies Appetizer With Breadcrumbs & Scallions,False,False,15,https://spoonacular.com/anchovies-appetizer-wi...,https://spoonacular.com/recipeImages/2-556x370...,1535,Starter,"anchovies,bread,garlic clove,olive oil,scallions",57.07,0.50,1,139.082615,1,2,1916
2,3,"Carrots, Cauliflower And Anchovies",True,False,45,https://spoonacular.com/carrots-cauliflower-an...,https://spoonacular.com/recipeImages/3-556x370...,1535,Maindish,"acini di pepe,almonds,anchovies,carrots,caulif...",942.08,11.09,4,107.376788,1,3,1918
3,4,Bap Story: Stir Fried Anchovies (Myulchi Bokkeum),True,False,45,https://spoonacular.com/bap-story-stir-fried-a...,https://spoonacular.com/recipeImages/4-556x370...,4245,Maindish,"anchovies,baby-back ribs,cooking oil,frying oi...",1697.23,14.09,3,112.312950,1,4,1919
4,5,"Bread, Butter And Anchovies",False,False,3,https://spoonacular.com/bread-butter-and-ancho...,https://spoonacular.com/recipeImages/5-556x370...,420,Maindish,"bread,capers,salted anchovies,unsalted butter",112.75,1.71,5,43.926995,1,5,1920


In [6]:
df['recommend'] = np.nan
df['recommend'] = df['recommend'].astype('str')

df["stars"] = df["stars"].astype("int64", errors='ignore')

for i in df['veryHealthy'].index: 
  if(False==df['veryHealthy'][i] ): 
    df['recommend'][i]='No' 
  elif(True==df['veryHealthy'][i] ): 
    df['recommend'][i]='Yes'

In [7]:
df.head()

,id,title,veryHealthy,veryPopular,readyInMinutes,spoonacularSourceUrl,image,aggregateLikes,dishTypes,ingredients,calories,Sugar/g,stars,popularity,userId,tfdbId,recipeId,recommend
0,1,Fried Anchovies with Sage,False,False,45,https://spoonacular.com/fried-anchovies-with-s...,https://spoonacular.com/recipeImages/1-556x370...,1354,Maindish,"anchovies,baking powder,egg,flour,sage leaves,...",384.09,0.17,3,150.437577,1,1,1915,No
1,2,Anchovies Appetizer With Breadcrumbs & Scallions,False,False,15,https://spoonacular.com/anchovies-appetizer-wi...,https://spoonacular.com/recipeImages/2-556x370...,1535,Starter,"anchovies,bread,garlic clove,olive oil,scallions",57.07,0.50,1,139.082615,1,2,1916,No
2,3,"Carrots, Cauliflower And Anchovies",True,False,45,https://spoonacular.com/carrots-cauliflower-an...,https://spoonacular.com/recipeImages/3-556x370...,1535,Maindish,"acini di pepe,almonds,anchovies,carrots,caulif...",942.08,11.09,4,107.376788,1,3,1918,Yes
3,4,Bap Story: Stir Fried Anchovies (Myulchi Bokkeum),True,False,45,https://spoonacular.com/bap-story-stir-fried-a...,https://spoonacular.com/recipeImages/4-556x370...,4245,Maindish,"anchovies,baby-back ribs,cooking oil,frying oi...",1697.23,14.09,3,112.312950,1,4,1919,Yes
4,5,"Bread, Butter And Anchovies",False,False,3,https://spoonacular.com/bread-butter-and-ancho...,https://spoonacular.com/recipeImages/5-556x370...,420,Maindish,"bread,capers,salted anchovies,unsalted butter",112.75,1.71,5,43.926995,1,5,1920,No


STAR COMPARE WITH RECOMMENDED

In [8]:
df.columns

Index(['id', 'title', 'veryHealthy', 'veryPopular', 'readyInMinutes',
       'spoonacularSourceUrl', 'image', 'aggregateLikes', 'dishTypes',
       'ingredients', 'calories', 'Sugar/g', 'stars', 'popularity', 'userId',
       'tfdbId', 'recipeId', 'recommend'],
      dtype='object')



*   Id : for food numbers
*   title : Food name
*   veryHealthy :
*   veryPopular :
*   readyInMinutes :
*   image :
*   dishTypes :
*   ingredients :
*   calories :
*   stars :
*   recommend :












In [9]:
df.iloc[0:3].transpose()

,0,1,2
id,1,2,3
title,Fried Anchovies with Sage,Anchovies Appetizer With Breadcrumbs & Scallions,"Carrots, Cauliflower And Anchovies"
veryHealthy,False,False,True
veryPopular,False,False,False
readyInMinutes,45,15,45
spoonacularSourceUrl,https://spoonacular.com/fried-anchovies-with-s...,https://spoonacular.com/anchovies-appetizer-wi...,https://spoonacular.com/carrots-cauliflower-an...
image,https://spoonacular.com/recipeImages/1-556x370...,https://spoonacular.com/recipeImages/2-556x370...,https://spoonacular.com/recipeImages/3-556x370...
aggregateLikes,1354,1535,1535
dishTypes,Maindish,Starter,Maindish
ingredients,"anchovies,baking powder,egg,flour,sage leaves,...","anchovies,bread,garlic clove,olive oil,scallions","acini di pepe,almonds,anchovies,carrots,caulif..."


In [10]:
df.shape

(1722, 18)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1722 entries, 0 to 1721
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1722 non-null   int64  
 1   title                 1722 non-null   object 
 2   veryHealthy           1722 non-null   bool   
 3   veryPopular           1722 non-null   bool   
 4   readyInMinutes        1722 non-null   int64  
 5   spoonacularSourceUrl  1722 non-null   object 
 6   image                 1718 non-null   object 
 7   aggregateLikes        1722 non-null   int64  
 8   dishTypes             1722 non-null   object 
 9   ingredients           1722 non-null   object 
 10  calories              1722 non-null   float64
 11  Sugar/g               1722 non-null   float64
 12  stars                 1722 non-null   int64  
 13  popularity            1722 non-null   float64
 14  userId                1722 non-null   int64  
 15  tfdbId               

In [12]:
# this is V
aggregateLike = df[df['aggregateLikes'].notnull()]['aggregateLikes'].astype('int')

# this is R
star = df[df['stars'].notnull()]['stars'].astype('int')

# this is C
C = star.mean()
C






3.9204413472706157

In [13]:
m = aggregateLike.quantile(0.95)
m

3233.8499999999995

In [14]:
qualified = df[(df['aggregateLikes'] >= m) & 
               (df['aggregateLikes'].notnull()) & 
               (df['stars'].notnull())][['title', 
                                                'calories', 
                                                'stars', 
                                                'aggregateLikes', 
                                                'popularity', 
                                                'dishTypes']]

qualified['aggregateLikes'] = qualified['aggregateLikes'].astype('int')
qualified['stars'] = qualified['stars'].astype('int')
qualified.shape

(87, 6)

In [15]:
def weighted_rating(x):
    v = x['aggregateLikes']
    R = x['stars']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [16]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [17]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [18]:
qualified.head(15)

,title,calories,stars,aggregateLikes,popularity,dishTypes,wr
190,Bagna Cauda Dip With Assorted Vegetables,1126.02,5,9635,89.938296,Starter,4.728715
1014,"Fish With Olives, Pine Nuts, Basil And Wine",26.01,5,9567,18.600490,Starter,4.727274
77,"Roasted Brussels Sprouts With Capers, Walnuts ...",611.24,5,6423,128.655964,Maindish,4.638481
1156,Bluefish (or Swordfish) With Preserved Lemons,371.76,5,5244,25.219805,Maindish,4.588206
489,"Spaghetti With Caramelized Onions, Anchovies, ...",1117.33,5,5235,10.706613,Maindish,4.587768
78,Labneh With Chilli And Anchovy,505.60,5,4646,94.199316,Maindish,4.556955
428,Salsa-Piqino,574.52,5,4645,59.113174,Maindish,4.556898
1587,Poached Salmon With Corn And White Wine–butter...,980.06,5,4524,30.380747,Maindish,4.549987
80,Pasta With Anchovy Bread Crumb Topping,650.02,5,4342,77.178973,Maindish,4.539176
988,What’s your sodium style,433.57,5,4254,3.314004,Maindish,4.533761


In [19]:
'''
>>> s
     a   b
one  1.  2.
two  3.  4.

>>> s.stack()
one a    1
    b    2
two a    3
    b    4
'''
s = df.apply(lambda x: pd.Series(x['dishTypes']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'dishTypes'
gen_df = df.drop('dishTypes', axis=1).join(s)
gen_df.head(3).transpose()

,0,1,2
id,1,2,3
title,Fried Anchovies with Sage,Anchovies Appetizer With Breadcrumbs & Scallions,"Carrots, Cauliflower And Anchovies"
veryHealthy,False,False,True
veryPopular,False,False,False
readyInMinutes,45,15,45
spoonacularSourceUrl,https://spoonacular.com/fried-anchovies-with-s...,https://spoonacular.com/anchovies-appetizer-wi...,https://spoonacular.com/carrots-cauliflower-an...
image,https://spoonacular.com/recipeImages/1-556x370...,https://spoonacular.com/recipeImages/2-556x370...,https://spoonacular.com/recipeImages/3-556x370...
aggregateLikes,1354,1535,1535
ingredients,"anchovies,baking powder,egg,flour,sage leaves,...","anchovies,bread,garlic clove,olive oil,scallions","acini di pepe,almonds,anchovies,carrots,caulif..."
calories,384.09,57.07,942.08


In [20]:
def build_chart(dishTypes, percentile=0.85):
    df = gen_df[gen_df['dishTypes'] == dishTypes]
    aggregateLikes = df[df['aggregateLikes'].notnull()]['aggregateLikes'].astype('int')
    stars = df[df['stars'].notnull()]['stars'].astype('int')
    C = stars.mean()
    m = aggregateLikes.quantile(percentile)
    
    qualified = df[(df['aggregateLikes'] >= m) & (df['aggregateLikes'].notnull()) & 
                   (df['stars'].notnull())][['title', 'ingredients', 'aggregateLikes', 'stars', 'popularity']]
    qualified['aggregateLikes'] = qualified['aggregateLikes'].astype('int')
    qualified['stars'] = qualified['stars'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: 
                        (x['aggregateLikes']/(x['aggregateLikes']+m) * x['stars']) + (m/(m+x['aggregateLikes']) * C),
                        axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [21]:
build_chart('Maindish').head(15)

,title,ingredients,aggregateLikes,stars,popularity,wr
77,"Roasted Brussels Sprouts With Capers, Walnuts ...","anchovies,brussels sprouts,capers,garlic clove...",6423,5,128.655964,4.792237
1156,Bluefish (or Swordfish) With Preserved Lemons,"basil leaves,brine,capers,grape tomatoes,lemon...",5244,5,25.219805,4.755676
489,"Spaghetti With Caramelized Onions, Anchovies, ...","bread,coarse salt,fresh flat-leaf parsley,garl...",5235,5,10.706613,4.755347
78,Labneh With Chilli And Anchovy,"anchovy fillets,bell pepper,bread,chilli,juice...",4646,5,94.199316,4.731731
428,Salsa-Piqino,"anchovy paste,basil,bay leaves,clams,dry white...",4645,5,59.113174,4.731687
1587,Poached Salmon With Corn And White Wine–butter...,"basil leaves,bay leaf,dry white wine,fresh cor...",4524,5,30.380747,4.726258
80,Pasta With Anchovy Bread Crumb Topping,"anchovies,cherry tomatoes,fresh parsley,garlic...",4342,5,77.178973,4.717665
988,What’s your sodium style,"crushed red pepper,dry white wine,fresh parsle...",4254,5,3.314004,4.713313
1000,Seafood Stew,"coarse salt,crusty bread,fennel bulb,garlic cl...",4244,5,30.387148,4.712810
219,Pasta With Anchovy-walnut side dish,"black pepper,black pepper,canned anchovy,crush...",4244,5,33.616115,4.712810


In [22]:
df.shape

(1722, 18)

In [23]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['title'])

tfidf_matrix.shape

(1722, 5465)

In [24]:
# http://scikit-learn.org/stable/modules/metrics.html#linear-kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

cosine_sim[0]
#cosine_sim.shape

array([1.        , 0.07178824, 0.0926404 , ..., 0.        , 0.        ,
       0.        ])

In [25]:
df = df.reset_index()
titles = df['title']
indices = pd.Series(df.index, index=df['title'])
#indices.head(2)

In [26]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    recipe_indices = [i[0] for i in sim_scores]
    return titles.iloc[recipe_indices]

In [27]:
get_recommendations('Garlic Parmesan Orange Roughy').head(10)

1616      Orange Roughy With Lemon, Garlic And Basil
1605             Orange Roughy With Citrus side dish
1607                Baked Orange Roughy With Veggies
1680                                   Parmesan Sole
322     Toasted Parmesan And Garlic Oil Caesar salad
1402                   Broiled Tilapia With Parmesan
298                        Parmesan & Anchovy Straws
1268               Whole Mackerel With Orange Butter
1516               Parmesan & Parsley-crusted Salmon
325                              Garlic Caesar salad
Name: title, dtype: object

In [28]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    recipes_indices = [i[0] for i in sim_scores]
    
    recipes = df.iloc[recipes_indices][['title', 'aggregateLikes', 'stars', 'image']]
    aggregateLike = recipes[recipes['aggregateLikes'].notnull()]['aggregateLikes'].astype('int')
    star = recipes[recipes['stars'].notnull()]['stars'].astype('int')
    C = star.mean()
    m = aggregateLike.quantile(0.60)
    qualified = recipes[(recipes['aggregateLikes'] >= m) & (recipes['aggregateLikes'].notnull()) & 
                       (recipes['stars'].notnull())]
    qualified['aggregateLikes'] = qualified['aggregateLikes'].astype('int')
    qualified['stars'] = qualified['stars'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [29]:
improved_recommendations('Fried Anchovies with Sage')

,title,aggregateLikes,stars,image,wr
1218,Cod With Sage Garlic Butter,1666,5,https://spoonacular.com/recipeImages/1397-556x...,4.287503
784,Pan-Fried Bass,1242,5,https://spoonacular.com/recipeImages/885-556x3...,4.220007
1192,Pan-fried Butterfish,4244,4,https://spoonacular.com/recipeImages/1367-556x...,3.965594
214,Corn Risotto Garnished with Bacon & Anchovy Sa...,2425,4,https://spoonacular.com/recipeImages/233-556x3...,3.954535
925,Pan Fried Sea Bass,2124,4,https://spoonacular.com/recipeImages/1052-556x...,3.951981
1404,Pan-fried Catfish,1434,4,https://spoonacular.com/recipeImages/1606-556x...,3.944882
763,Pan Fried Striped Bass,1313,4,https://spoonacular.com/recipeImages/862-556x3...,3.943416
815,Pan-Fried Striped Bass with Stir-Fried Tomatoe...,1242,3,https://spoonacular.com/recipeImages/918-556x3...,3.665029
3,Bap Story: Stir Fried Anchovies (Myulchi Bokkeum),4245,3,https://spoonacular.com/recipeImages/4-556x370...,3.397998
1525,Prosciutto-wrapped Halibut With Sage Butter si...,3453,2,https://spoonacular.com/recipeImages/1756-556x...,2.928751


In [30]:
improved_recommendations('Carrots, Cauliflower And Anchovies')

,title,aggregateLikes,stars,image,wr
32,Roasted Cauliflower With Anchovy Bread Crumbs,1324,5,https://spoonacular.com/recipeImages/37-556x37...,4.234040
41,"Orecchiette with Cauliflower, Anchovies and Pi...",3735,4,https://spoonacular.com/recipeImages/46-556x37...,3.963081
1213,Baked Flounder With Parsnips And Carrots Recipe,1314,4,https://spoonacular.com/recipeImages/1392-556x...,3.943428
845,Sautéed Bass With Mint Pesto And Spiced Carrots,1242,4,https://spoonacular.com/recipeImages/951-556x3...,3.942518
1114,Steamed White Fish With Julienned Carrots And ...,1124,4,https://spoonacular.com/recipeImages/1269-556x...,3.940962
102,Roasted Cauliflower With Lemon And Anchovies,1111,4,https://spoonacular.com/recipeImages/110-556x3...,3.940785
836,Striped Bass with Sweet Carrots and Cider Glaze,1242,3,https://spoonacular.com/recipeImages/940-556x3...,3.665029
1499,Sake-Steamed Halibut with Dilled Carrots,1323,3,https://spoonacular.com/recipeImages/1723-556x...,3.653208
825,Seared Bass with Cauliflower Duo,1242,2,https://spoonacular.com/recipeImages/928-556x3...,3.387540
258,Roasted Salmon With Spicy Cauliflower,6766,3,https://spoonacular.com/recipeImages/282-556x3...,3.297661


CF based recommendation system

In [31]:
!pip install scikit-surprise
!pip install evaluate


     |████████████████████████████████| 11.8 MB 3.8 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630175 sha256=7a714635408f50c39a396a4414d0a9c5ea786c5376c690e48760a13baad9d98f
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise
  Created wheel for evaluate: filename=evaluate-0.0.3-py3-none-any.whl size=6860 sha256=7f9a563eb970c68d0836ae46f48be61cc6feafd860d1cc4ec18f94a3bafa5c9b
  Stored in directory: /root/.cache/pip/wheels/0e/81/86/5dd2648f1ca66017709d332ea7251e5443fac671670ee487a2
Successfully built evaluate


In [32]:
#Importing surprise reader
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate


In [33]:
# surprise reader API to read the dataset
reader = Reader()

In [34]:
data = Dataset.load_from_df(df[['id','userId','stars']], reader)


In [35]:
svd = SVD()

cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9638  0.9252  0.9495  0.9880  0.9633  0.9580  0.0205  
MAE (testset)     0.6761  0.6953  0.6785  0.7188  0.7207  0.6979  0.0190  
Fit time          0.10    0.08    0.09    0.09    0.09    0.09    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'fit_time': (0.09586262702941895,
  0.0841364860534668,
  0.09470129013061523,
  0.08576035499572754,
  0.08603787422180176),
 'test_mae': array([0.67611127, 0.69530595, 0.67850794, 0.71875106, 0.72068563]),
 'test_rmse': array([0.96382753, 0.92519845, 0.94952244, 0.98796274, 0.96329383]),
 'test_time': (0.002231597900390625,
  0.0020732879638671875,
  0.00429224967956543,
  0.0020780563354492188,
  0.0020546913146972656)}

In [36]:

data_train = data.build_full_trainset()
svd.fit(data_train)

In [37]:
df[df['userId'] == 1]

,index,id,title,veryHealthy,veryPopular,readyInMinutes,spoonacularSourceUrl,image,aggregateLikes,dishTypes,ingredients,calories,Sugar/g,stars,popularity,userId,tfdbId,recipeId,recommend
0,0,1,Fried Anchovies with Sage,False,False,45,https://spoonacular.com/fried-anchovies-with-s...,https://spoonacular.com/recipeImages/1-556x370...,1354,Maindish,"anchovies,baking powder,egg,flour,sage leaves,...",384.09,0.17,3,150.437577,1,1,1915,No
1,1,2,Anchovies Appetizer With Breadcrumbs & Scallions,False,False,15,https://spoonacular.com/anchovies-appetizer-wi...,https://spoonacular.com/recipeImages/2-556x370...,1535,Starter,"anchovies,bread,garlic clove,olive oil,scallions",57.07,0.50,1,139.082615,1,2,1916,No
2,2,3,"Carrots, Cauliflower And Anchovies",True,False,45,https://spoonacular.com/carrots-cauliflower-an...,https://spoonacular.com/recipeImages/3-556x370...,1535,Maindish,"acini di pepe,almonds,anchovies,carrots,caulif...",942.08,11.09,4,107.376788,1,3,1918,Yes
3,3,4,Bap Story: Stir Fried Anchovies (Myulchi Bokkeum),True,False,45,https://spoonacular.com/bap-story-stir-fried-a...,https://spoonacular.com/recipeImages/4-556x370...,4245,Maindish,"anchovies,baby-back ribs,cooking oil,frying oi...",1697.23,14.09,3,112.312950,1,4,1919,Yes
4,4,5,"Bread, Butter And Anchovies",False,False,3,https://spoonacular.com/bread-butter-and-ancho...,https://spoonacular.com/recipeImages/5-556x370...,420,Maindish,"bread,capers,salted anchovies,unsalted butter",112.75,1.71,5,43.926995,1,5,1920,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,170,182,Grilled Kale Caesar salad,False,False,45,https://spoonacular.com/grilled-kale-caesar-Sa...,https://spoonacular.com/recipeImages/182-556x3...,635,Salad,"anchovy,black pepper,croutons,dijon mustard,ga...",440.07,1.09,4,39.604363,1,182,1827,No
171,171,183,Anchovy Mayonnaise,False,False,30,https://spoonacular.com/anchovy-mayonnaise-183,https://spoonacular.com/recipeImages/183-556x3...,635,Sidedish,"anchovies,garlic cloves,mayonnaise,milk,olive oil",219.54,2.13,4,36.973031,1,183,1829,No
172,172,184,Anchovy Fennel Toasts with Roasted Red Peppers,False,False,45,https://spoonacular.com/anchovy-fennel-toasts-...,https://spoonacular.com/recipeImages/184-556x3...,635,Starter,"anchovy,baguette,black pepper,fennel seeds,lem...",39.72,0.27,5,99.687084,1,184,1830,No
173,173,185,Double-whammy Arrabiata,True,False,32,https://spoonacular.com/double-whammy-arrabiat...,https://spoonacular.com/recipeImages/185-556x3...,29,Maindish,"anchovy,bread,chilli,chillies,dried chilli,gar...",1737.53,24.35,4,17.773500,1,185,1831,Yes


In [38]:
svd.predict(1, 302)

Prediction(uid=1, iid=302, r_ui=None, est=3.819796868214354, details={'was_impossible': False})

Hybrid recommendation system

In [39]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [40]:
id_map = pd.read_csv('/content/newfood.csv')[['recipeId', 'tfdbId']]
id_map['tfdbId'] = id_map['tfdbId'].apply(convert_int)
id_map.columns = ['recipeId', 'id']
id_map = id_map.merge(df[['title', 'id']], on='id').set_index('title')

In [41]:
indices_map = id_map.set_index('id')

In [42]:
def hybrid(userId,title):
    idx = df[df.title==title].index.values[0]
    tfdbId = df[df.title==title]['tfdbId'].values[0]
    recipe_id = df[df.title==title]['recipeId'].values[0]
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    recipe_indices = [i[0] for i in sim_scores]
    recipes = df.iloc[recipe_indices][['title','ingredients','id','recipeId']]

    recipes['est'] = recipes['recipeId'].apply(lambda x: svd.predict(userId, x).est)

    recipes = recipes.sort_values('est', ascending=False)
    return recipes.head(10)

In [43]:
hybrid(2,"Pissaladiere")

,title,ingredients,id,recipeId,est
327,Pissaladiere,"all purpose flour,anchovy,bay leaf,capers,fres...",362,161,3.654128
8,Marinated Boquerones,"bell pepper,chillies,coarse sea salt,extra-vir...",10,1924,3.654128
18,Anchovy And Onion Tart,"active yeast,anchovy filets,bouquet garni,flou...",22,1936,3.654128
17,"Anchovies, Beans And Rocket","anchovies in extra virgin olive oil,borlotti b...",21,1935,3.654128
16,"Pan-roasted East Coast Skate, Anchovy And Broc...","capers,extra virgin olive oil,garlic,green chi...",20,1934,3.654128
15,No-knead Fennel & Anchovy Pizza,"anchovy,cornmeal,fennel,fennel bulb,flour,lemo...",19,1933,3.654128
14,Anchovy Vinaigrette,"bell pepper,club soda,cream-style corn,dijon m...",18,1932,3.654128
13,Savory Slow Roasted Tomatoes with Filet of Anc...,"anchovy filets,extra virgin olive oil,fire-roa...",17,1930,3.654128
12,Sliced Baguette With Radishes And Anchovy Butter,"anchovy,baguette,chives,coarse kosher salt,fre...",16,1929,3.654128
11,"Anchovies, Tomatoes And Green Beans Couscous","anchovies,anchovies in extra virgin olive oil,...",13,1928,3.654128
